In [ ]:
import time
from dill import dump_session, load_session
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection, metrics
from toolbox_02450 import mcnemar
import scipy.stats as st

start_time = time.time()

### One-hot-encoding for categorical attributes

In [ ]:
# Read file and store as pandas dataframe
filename = '../Data/day.csv'
df = pd.read_csv(filename)

# dropped_attributes = ['instant','dteday','yr','mnth','holiday','weekday','casual','registered']
dropped_attributes = ['instant','dteday','yr','mnth','holiday','weekday']
for attribute in dropped_attributes:
    df = df.drop(attribute, axis=1)
    
# One hot encoding
ohe_df = pd.get_dummies(df, columns = ['season','weathersit'])
ohe_df = ohe_df.drop(['season_4','weathersit_3'], axis=1)    # season_4, weathersit_3 are chosen as reference variables
display(ohe_df)

### Standardize data

In [ ]:
N, M = ohe_df.shape
attribute_names = list(ohe_df.columns)

# Get column indexes
temp_col = ohe_df.columns.get_loc("temp")
atemp_col = ohe_df.columns.get_loc("atemp")
hum_col = ohe_df.columns.get_loc("hum")
wspd_col = ohe_df.columns.get_loc("windspeed")
cnt_col = ohe_df.columns.get_loc("cnt")

# Undo the original max-min normalization
data = ohe_df.values
for row in range(0, N):
    data[row, temp_col] = data[row, temp_col]*(39-(-8)) + (-8)
    data[row, atemp_col] = data[row, atemp_col]*(50-(-16)) + (-16)
    data[row, hum_col] = data[row, hum_col]*100
    data[row, wspd_col] = data[row, wspd_col]*67

# Standarize ratio data attributes
for col in range(temp_col, cnt_col+1): # subtract mean column-wise
    mn = data[:, col].mean(0)
    std = np.std(data[:, col])
    data[:, col] = (data[:, col] - np.ones(N)*mn)/std

# Create DataFrame for visualisation
data_df = pd.DataFrame(data, columns=attribute_names)
display(data_df)

### Set 'workingday' as target variable

In [ ]:
# Split dataset into features and target vector
workingday = attribute_names.index("workingday")
y = data[:,workingday]
X = np.delete(data, workingday, axis=1)
attribute_names.pop(workingday)
N, M = X.shape

# Add offset attribute
X = np.concatenate((np.ones((X.shape[0],1)),X),1)
attribute_names = [u'offset']+attribute_names
M = M+1

print("N: {}, M: {} (including offset)".format(N,M))

In [ ]:
attribute_names

### 1. Regularized Logistic Regression

In [ ]:
# Values of lambda
lambdas = np.power(10.,np.linspace(-5,9,100))

### 2. K-Nearest-Neighbours 

In [ ]:
# Number of neighbours
k_start = 1
k_end = 100
k_values = list(range(k_start,k_end+1))
dist = 2
metric = 'minkowski'
metric_params = {}  # no parameters needed for minkowski

### Cross Validation

In [ ]:
# Cross validation
K_outer = 10
K_inner = 10

cv_outer = model_selection.KFold(K_outer, shuffle=True, random_state=7)
cv_inner = model_selection.KFold(K_inner, shuffle=True, random_state=7)

In [ ]:
# Initialize variables
Error_test_rlr = np.empty((K_outer,1))
Error_test_base = np.empty((K_outer,1))
opt_lambdas = np.empty((K_outer,1))             # optimal lambdas for each outer fold

Error_test_KNN = np.empty((K_outer,1))
opt_k_values = np.empty((K_outer,1))             # optimal neighbours for each outer fold

mu = np.empty((K_outer, M-1))
sigma = np.empty((K_outer, M-1))

# Statistical evaluation
y_hat = np.zeros((1,3))       # estimates for each model [KNN, rlr, baseline], to delete 1st row at the end
y_true = np.zeros((1))

### Training and Testing

In [ ]:
for k_out, (par_index, test_index) in enumerate(cv_outer.split(X,y)):
    
    print('\nOuter Cross Validation Fold: {0}/{1}'.format(k_out+1,K_outer))
    
    # Split outer fold into parameterisation set and test set
    X_par = X[par_index]
    y_par = y[par_index].astype(int)
    X_test = X[test_index]
    y_test = y[test_index].astype(int)
    y_par = y_par.squeeze()
    
    # Initialise error arrays
    w = np.empty((M,K_inner,len(lambdas)))           
    lr_train_error = np.empty((K_inner,len(lambdas)))
    lr_val_error = np.empty((K_inner,len(lambdas)))    
    coefficient_norm = np.empty((K_inner,len(lambdas)))   
    
    knn_train_error = np.empty((K_inner,len(k_values)))
    knn_val_error = np.empty((K_inner,len(k_values)))    
    
    for k_in, (train_index, val_index) in enumerate(cv_inner.split(X_par,y_par)):
    
        print('\n\tInner Fold: {}/{}'.format(k_in+1,K_inner))
        # Split parameterisation set into training set and validation set
        X_train = X[train_index]
        y_train = y[train_index].astype(int)
        X_val = X[val_index]
        y_val = y[val_index].astype(int)
        
        # Standardize inner fold based on training set, and save the mean and std
        mu_train = np.mean(X_train[:, 1:], 0)
        sigma_train = np.std(X_train[:, 1:], 0)
        X_train[:, 1:] = (X_train[:, 1:] - mu_train) / sigma_train
        X_val[:, 1:] = (X_val[:, 1:] - mu_train) / sigma_train
    
        ########################### Logistic Regression ###########################
        # solve for weights
        for l in range(0,len(lambdas)):
            mdl = LogisticRegression(penalty='l2', C=1/lambdas[l] )
            mdl.fit(X_train, y_train)
            w[:,k_in,l] = mdl.coef_[0]
            # w[:,k_in,l] = np.sqrt(np.sum(w_est**2))
            
            y_train_est = mdl.predict(X_train).T.astype(int)
            y_val_est = mdl.predict(X_val).T.astype(int)
                    
            # Evaluate training and validation performance
            lr_train_error[k_in,l] = np.sum(y_train_est != y_train) / len(y_train)
            lr_val_error[k_in,l] = np.sum(y_val_est != y_val) / len(y_val)
        
        ################################ KNN #################################
        for k_i in range(0,len(k_values)):
            knclassifier = KNeighborsClassifier(n_neighbors=k_values[k_i], p=dist,
                                    metric=metric,
                                    metric_params=metric_params)
            knclassifier.fit(X_train, y_train)
            
            y_train_est = knclassifier.predict(X_train)
            y_val_est = knclassifier.predict(X_val)
            
            # Evaluate training and validation performance
            knn_train_error[k_in,k_i] = np.sum(y_train_est != y_train) / len(y_train)
            knn_val_error[k_in,k_i] = np.sum(y_val_est != y_val) / len(y_val)

    ########################### Logistic Regression ###########################
    opt_val_err = np.min(lr_val_error)
    opt_lambda = lambdas[np.argmin(np.mean(lr_val_error,axis=0))]
    opt_lambdas[k_out] = opt_lambda
    print("Optimal lambda found: ", round(np.log10(opt_lambda),5))
   
    train_err_vs_lambda = np.mean(lr_train_error,axis=0)
    val_err_vs_lambda = np.mean(lr_val_error,axis=0)
    mean_w_vs_lambda = np.squeeze(np.mean(w,axis=1))

    # Retrain logistic regression model on parameterisation set using optimal lambda
    mdl = LogisticRegression(penalty='l2', C=1/opt_lambda )
    mdl.fit(X_par, y_par)

    # y_par_est = mdl.predict(X_par).T.astype(int)
    y_test_est = mdl.predict(X_test).T.astype(int)

    # Evaluate test performance
    Error_test_rlr[k_out] = np.sum(y_test_est != y_test) / len(y_test)
    
    ########################### Baseline Model ###########################
    majority_class = np.bincount(y_test).argmax()
    Error_test_base[k_out] = np.sum(y_test != majority_class) / len(y_test)
    
    # For statistical evaluation
    y_hat_rlr = np.expand_dims(y_test_est, axis=1)
    y_hat_base = np.full((y_test.shape[0], 1), majority_class)
    y_true = np.append(y_true, y_test, axis=0)
    
    ################################ KNN #################################
    opt_k_err = np.min(knn_val_error)
    opt_k = k_values[np.argmin(np.mean(knn_val_error,axis=0))]
    opt_k_values[k_out] = opt_k
    print("Optimal k found: ", opt_k)
    
    # Retrain KNN model on parameterisation set using optimal lambda
    knclassifier = KNeighborsClassifier(n_neighbors=opt_k, p=dist,
                                    metric=metric,
                                    metric_params=metric_params)
    knclassifier.fit(X_par, y_par)
    y_test_est = knclassifier.predict(X_test)
    
    # Evaluate test performance
    Error_test_KNN[k_out] = np.sum(y_test_est != y_test) / len(y_test)
    
    # For statistical evaluation
    y_hat_knn = np.expand_dims(y_test_est, axis=1)
    y_hat_fold = np.concatenate((y_hat_knn, y_hat_rlr, y_hat_base), axis=1)
    y_hat = np.append(y_hat, y_hat_fold, axis=0)
    
   # Display the results for the last outer cross-validation fold
    if k_out == K_outer-1:
        reg_fig, reg_axes = plt.subplots(1, 2, figsize=(10,5),num=1)
        reg_axes[0].semilogx(lambdas,mean_w_vs_lambda.T[:,1:],'-',label=attribute_names[1:]) # Don't plot the bias term
        reg_axes[0].set_xlabel('Regularization factor')
        reg_axes[0].set_ylabel('Mean Coefficient Values')
        reg_axes[0].grid()
        reg_axes[0].legend()

        reg_axes[1].set_title('Optimal lambda: 1e{0}'.format(round(np.log10(opt_lambdas[k_out][0]),5)))
        reg_axes[1].loglog(lambdas,train_err_vs_lambda.T,'b-',lambdas,val_err_vs_lambda.T,'r-')
        reg_axes[1].set_xlabel('Regularization factor')
        reg_axes[1].set_ylabel('Squared error (crossvalidation)')
        reg_axes[1].legend(['Train error','Validation error'])
        reg_axes[1].grid()
        reg_fig.tight_layout()        
        
# For statistical evaluation
y_hat = np.delete(y_hat, 0, 0)      # delete the 1st row of zeros
y_true = np.delete(y_true, 0, 0)      # delete the 1st row of zeros


### Plot Weights from Logistic Regression

In [ ]:
# # Bar plot of RLR weights
# weights = range(1,M)      # skip offset
# bw = 1.0/(len(weights)+1)
# r = np.arange(1,K_outer+1)

# plt.figure(figsize=(6,6), num=3)
# for i in weights:
#     plt.bar(r+i*bw, w[i,:], width=bw)
# plt.xticks(r+bw, range(1,K_outer+1))
# plt.xlabel('Attributes')
# plt.ylabel('Weights')
# plt.legend(attribute_names[1:M+1], loc=(1.04, 0))
# plt.grid()
# plt.title('Weights from each fold of Regularized Linear Regression')

### Table 2

In [ ]:
print("Outer fold \tKNN \t\t\tLogistic Regression \t\tBaseline")
print("--------------------------------------------------------------------------------")
print("i \t\tk_i \tEtest_i \tlambda_i \tEtest_i \tEtest_i")
for i in range(K_outer):
    print("{} \t\t{:.3f} \t{:.3f} \t\t1e{} \t{:.3f} \t\t{:.3f}".format(\
        i+1,opt_k_values[i][0],Error_test_KNN[i].item(),round(np.log10(opt_lambdas[i][0]),3),\
            Error_test_rlr[i][0],Error_test_base[i][0]))

## Statistical Evaluation

In [ ]:
def print_mcnemar(thetahat, CI, p):
    print("\t CI : ({:.5f}, {:.5f})".format(CI[0], CI[1]))
    print("\t p: {:.5e}".format(p))
    print("\t theta_hat: {:.5f}\n".format(thetahat))    


In [ ]:
# [KNN, RLR, Baseline]
alpha = 0.05
[thetahat_1, CI_1, p_1] = mcnemar(y_true, y_hat[:,0], y_hat[:,1], alpha=alpha)    # KNN vs RLR
print("\n")
[thetahat_2, CI_2, p_2] = mcnemar(y_true, y_hat[:,0], y_hat[:,2], alpha=alpha)    # KNN vs Baseline
print("\n")
[thetahat_3, CI_3, p_3] = mcnemar(y_true, y_hat[:,1], y_hat[:,2], alpha=alpha)    # RLR vs Baseline
print("\n")

print("\n")
print_mcnemar(thetahat_1, CI_1, p_1)    # RLR better than KNN
print_mcnemar(thetahat_2, CI_2, p_2)    # KNN better than baseline
print_mcnemar(thetahat_3, CI_3, p_3)    # RLR better than baseline

# positive theta_hat means A better than B

In [ ]:
print("KNN Confusion Matrix")
print(metrics.confusion_matrix(y_true, y_hat[:,0]))

print("\nRegularised Logistic Regression Confusion Matrix")
print(metrics.confusion_matrix(y_true, y_hat[:,1]))

print("\nBaseline Confusion Matrix")
print(metrics.confusion_matrix(y_true, y_hat[:,2]))

In [ ]:
accuracy = [np.sum(y_hat[:,i]==y_true)/len(y_true) for i in range(y_hat.shape[1])]
accuracy_raw = [np.sum(y_hat[:,i]==y_true) for i in range(y_hat.shape[1])]

print("accuracy %: ", accuracy)
print("accuracy raw: ", accuracy_raw)

## Housekeeping

In [ ]:
elapsed_time = time.time() - start_time
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

# dump_session('sun_night_2_env.db')
# dill.load_session('friday_2234_env.db')

## Saved Results Below